In [1]:
import sqlite3
import pandas as pd
import numpy as np

In [2]:
conn = sqlite3.connect(":memory:")
cur = conn.cursor()

In [3]:
cur.executescript("""
DROP TABLE IF EXISTS teams;
DROP TABLE IF EXISTS players;
DROP TABLE IF EXISTS matches;

CREATE TABLE teams (
  team    TEXT PRIMARY KEY,
  city    TEXT NOT NULL,
  founded INTEGER NOT NULL
);

CREATE TABLE players (
  player_id INTEGER PRIMARY KEY AUTOINCREMENT,
  name   TEXT NOT NULL,
  team   TEXT NOT NULL REFERENCES teams(team),
  pos    TEXT NOT NULL,
  age    INTEGER NOT NULL,
  goals  INTEGER NOT NULL,
  assists INTEGER,
  minutes INTEGER NOT NULL
);

CREATE TABLE matches (
  match_id   INTEGER PRIMARY KEY AUTOINCREMENT,
  date       TEXT NOT NULL,
  home       TEXT NOT NULL,
  away       TEXT NOT NULL,
  home_goals INTEGER NOT NULL,
  away_goals INTEGER NOT NULL
);

INSERT INTO teams(team, city, founded) VALUES
 ('Lions','Sofia', 2015),
 ('Wolves','Plovdiv',1914),
 ('Eagles','Varna',1930);

INSERT INTO players(name, team, pos, age, goals, assists, minutes) VALUES
 ('Ivan Petrov','Lions','FW',24,11,3,1350),
 ('Martin Kolev','Lions','MF',29,4,NULL,1490),
 ('Rui Costa','Lions','DF',31,1,2,1600),
 ('Georgi Iliev','Wolves','FW',27,7,5,1410),
 ('Joe Jackson','Wolves','FW',27,17,5,410),
 ('Peter Marin','Eagles','FW',20,5,1,870);

INSERT INTO matches(date,home,away,home_goals,away_goals) VALUES
 ('2024-08-03','Lions','Wolves',2,1),
 ('2024-08-10','Eagles','Lions',1,3),
 ('2024-08-17','Wolves','Eagles',2,2);
""")
conn.commit()

In [4]:
teams = pd.read_sql("SELECT * FROM teams", conn)
players = pd.read_sql("SELECT * FROM players", conn)
matches = pd.read_sql("SELECT * FROM matches", conn, parse_dates = ["date"])

In [5]:
players.head()

,player_id,name,team,pos,age,goals,assists,minutes
0,1,Ivan Petrov,Lions,FW,24,11,3.0,1350
1,2,Martin Kolev,Lions,MF,29,4,NaN,1490
2,3,Rui Costa,Lions,DF,31,1,2.0,1600
3,4,Georgi Iliev,Wolves,FW,27,7,5.0,1410
4,5,Joe Jackson,Wolves,FW,27,17,5.0,410


In [6]:
teams.head()#, matches.head(), players.head()

,team,city,founded
0,Lions,Sofia,2015
1,Wolves,Plovdiv,1914
2,Eagles,Varna,1930


In [7]:
#SQL vs Pandas - Filtering

In [8]:
sql1 = pd.read_sql("""
SELECT name, team, goals
FROM players
WHERE pos = 'FW' AND minutes > 1200;
""", conn)                   

In [9]:
sql1

,name,team,goals
0,Ivan Petrov,Lions,11
1,Georgi Iliev,Wolves,7


In [10]:
pd1 = players.loc[(players['pos']=='FW')&(players["minutes"]>1200),["name", "team", "goals"]]

In [11]:
pd1

,name,team,goals
0,Ivan Petrov,Lions,11
3,Georgi Iliev,Wolves,7


In [12]:
# Aggregation - Total goals per team

In [13]:
sql2 = pd.read_sql("""
SELECT team, SUM(goals)
FROM players
GROUP BY team
ORDER BY 2 DESC;
""", conn)                   

In [14]:
sql2

,team,SUM(goals)
0,Wolves,24
1,Lions,16
2,Eagles,5


In [15]:
pd2 = players.groupby("team")["goals"].sum().reset_index()

In [16]:
pd2.sort_values("goals", ascending = False).reset_index(drop=True)

,team,goals
0,Wolves,24
1,Lions,16
2,Eagles,5


In [17]:
# Join and Merge

In [18]:
sql3 = pd.read_sql("""
SELECT p.name, t.city
FROM players p
JOIN teams t on t.team = p.team;
""", conn)                   

In [19]:
sql3

,name,city
0,Ivan Petrov,Sofia
1,Martin Kolev,Sofia
2,Rui Costa,Sofia
3,Georgi Iliev,Plovdiv
4,Joe Jackson,Plovdiv
5,Peter Marin,Varna


In [20]:
pd3 = players.merge(teams, on="team", how="left")

In [21]:
pd3[["name", "city"]]

,name,city
0,Ivan Petrov,Sofia
1,Martin Kolev,Sofia
2,Rui Costa,Sofia
3,Georgi Iliev,Plovdiv
4,Joe Jackson,Plovdiv
5,Peter Marin,Varna


In [22]:
m = matches.copy()

In [23]:
m

,match_id,date,home,away,home_goals,away_goals
0,1,2024-08-03,Lions,Wolves,2,1
1,2,2024-08-10,Eagles,Lions,1,3
2,3,2024-08-17,Wolves,Eagles,2,2


In [24]:
m["home_points"] = np.where(m["home_goals"]>m["away_goals"],3,
                     np.where(m["home_goals"]==m["away_goals"],1,0))
m["away_points"] = np.where(m["away_goals"]>m["home_goals"],3,
                     np.where(m["away_goals"]==m["home_goals"],1,0))

home_tbl = m[["home","home_points","home_goals","away_goals"]] \
              .rename(columns={"home":"team","home_points":"points","home_goals":"gf","away_goals":"ga"})
away_tbl = m[["away","away_points","away_goals","home_goals"]] \
              .rename(columns={"away":"team","away_points":"points","away_goals":"gf","home_goals":"ga"})

total_points = pd.concat([home_tbl,away_tbl])
league = total_points.groupby("team").agg(points=("points","sum"), GF=("gf","sum"), GA=("ga","sum"))
league["GD"] = league["GF"] - league["GA"]
league.sort_values(["points","GD"], ascending=[False,False])

,points,GF,GA,GD
team,,,,
Lions,6,5,2,3
Wolves,1,3,4,-1
Eagles,1,3,5,-2


In [25]:
pd4 = players.nlargest(3, "goals")
pd4 = pd4[["name", "team", "goals"]]
pd4

,name,team,goals
4,Joe Jackson,Wolves,17
0,Ivan Petrov,Lions,11
3,Georgi Iliev,Wolves,7


In [26]:
bins = [0, 22, 26, 30, np.inf]
labels = ["<=22", "23-26", "27-30", "31+"]

In [27]:
players["age_band"] = pd.cut(players["age"], bins = bins, labels = labels)

In [28]:
players["age_band"].value_counts().reset_index()

,age_band,count
0,27-30,3
1,<=22,1
2,23-26,1
3,31+,1
